In [29]:
import torch

from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim import Adam
from torchvision.datasets import MNIST
from ignite.engine.engine import Events, Engine

import numpy as np
import cv2
import matplotlib.pyplot as plt

import math
from pathlib import Path
import pprint

from model import get_critic, get_generator

In [30]:
class InfiniteDataLoader:
    def __init__(self, data_loader):
        self.data_loader = data_loader

    @property
    def batch_size(self):
        return self.data_loader.batch_size

    def __iter__(self):
        self.iterator = iter(self.data_loader)
        return self

    def __next__(self):
        try:
            batch = next(self.iterator)
        except StopIteration:
            self.iterator = iter(self.data_loader)
            batch = next(self.iterator)

        return batch


In [31]:
class WGANTrainer:
    def __init__(self, data_loader, gen: nn.Module, cri: nn.Module,
                 opt_g, opt_c, n_cri, gp_lam: float, device):

        assert len(n_cri) == 2

        self.data_loader = data_loader
        self.gen = gen
        self.cri = cri
        self.opt_g = opt_g
        self.opt_c = opt_c
        self.n_cri = n_cri
        self.gp_lam = gp_lam
        self.device = device

    def __call__(self, engine, batch):
        # 複数回サンプリングする必要があるため渡されたbatchは使わない
        # データはself.data_loaderから取得

        self.gen.train()
        self.cri.train()

        # update critic
        iter_num = engine.state.iteration
        upd_num = self.n_cri[1] if iter_num <= 25 or iter_num % 500 == 0 else self.n_cri[0]
        critic_mean_loss = 0
        i = 0
        for batch in self.data_loader:
            x_real, _ = batch
            x_real = x_real.to(self.device)

            z = np.random.uniform(size=(self.data_loader.batch_size, Z_DIM)).astype(np.float32)
            z = torch.from_numpy(z).to(self.device)
            x_fake = self.gen(z).detach()
            x_fake.requires_grad = True

            cri_loss = (self.cri(x_fake) - self.cri(x_real)).mean()  # Wasserstein距離の逆符号

            # gradient penalty
            eps = np.random.uniform(size=(self.data_loader.batch_size, 1, 1, 1)).astype(np.float32)
            eps = torch.from_numpy(eps).to(self.device)
            x_fusion = eps * x_real + (1 - eps) * x_fake  # (N,1,H,W)
            g_critic = torch.autograd.grad(torch.unbind(self.cri(x_fusion)), x_fusion, create_graph=True)[0]  # (N,1,H,W)
            gp = g_critic.square().sum(dim=(1, 2, 3)).sqrt()
            gp = (gp - 1).square().mean()

            total_loss = cri_loss + self.gp_lam * gp

            self.opt_c.zero_grad()
            total_loss.backward()
            self.opt_c.step()

            critic_mean_loss += total_loss.item()

            i += 1
            if i >= upd_num:
                break

        critic_mean_loss /= upd_num # equivalente a: critic_mean_loss = critic_mean_loss / upd_num

        # update generator
        z = np.random.uniform(size=(self.data_loader.batch_size, Z_DIM)).astype(np.float32)
        z = torch.from_numpy(z).to(self.device)
        x_fake = self.gen(z)
        gen_loss = -self.cri(x_fake).mean()

        self.opt_c.zero_grad()
        self.opt_g.zero_grad()
        gen_loss.backward()
        self.opt_g.step()

        return {
            "critic_loss": critic_mean_loss,
            "generator_loss": gen_loss.item()
        }


class MetricsAccumulator:
    def __init__(self, keys: list):
        self.keys = keys
        self.logs = {k: 0.0 for k in keys}
        self.num = 0

    def __call__(self, engine):
        for k in self.keys:
            self.logs[k] += engine.state.output[k]
        self.num += 1

    def get_mean(self, key):
        return self.logs[key] / self.num

    def reset(self):
        self.logs = {k: 0.0 for k in self.keys}
        self.num = 0

In [32]:
def record_metrics(log_dict: dict, accumulator: MetricsAccumulator, keys: list = None):
    if keys is None:
        keys = accumulator.keys

    def _record(engine):
        _add("iteration", engine.state.iteration)

        for k in keys:
            _add(k, accumulator.get_mean(k))

        accumulator.reset()

    def _add(key, value):
        if key in log_dict:
            log_dict[key].append(value)
        else:
            log_dict[key] = [value]

    return _record


def plot_metrics(log_dict: dict, x_key: str, y_keys: list, out_path: Path):
    def _plot(engine):
        plt.figure()
        for k in y_keys:
            plt.plot(log_dict[x_key], log_dict[k], label=k)
        plt.legend()
        plt.xlabel(x_key)
        plt.savefig(str(out_path))
        plt.close()

    return _plot


def print_metrics(log_dict: dict, keys: list):
    return lambda engine: print(", ".join(f"{k}: {log_dict[k][-1]}" for k in keys))


def save_img(generator: nn.Module, out_dir_path: Path, device):
    try:
        out_dir_path.mkdir(parents=True)
    except FileExistsError:
        pass

    def _save(engine):
        z = np.random.uniform(size=(1, Z_DIM)).astype(np.float32)
        z = torch.from_numpy(z).to(device)

        generator.eval()
        with torch.no_grad():
            x = generator(z).detach().cpu().numpy().squeeze() * 255

        p = out_dir_path / f"out_iter_{engine.state.iteration:05d}.png"
        cv2.imwrite(str(p), x.astype(np.uint8))

    return _save



In [33]:
Z_DIM = 30
def main(args):
    # Adaptado para usar aceleración en MacOS
    if args.g >= 0:
        if torch.cuda.is_available():
            device = torch.device(f"cuda:{args.g:d}")
            print(f"GPU CUDA mode: {args.g:d}")
        if torch.backends.mps.is_available():
            device = torch.device("mps")
            print(f"GPU mps for MacOS")
    else:
        device = torch.device("cpu")
        print("CPU mode")

    result_dir = Path(args.result_dir)

    # MNISTデータ取得
    mnist_train = MNIST(root=".", download=True, train=True,
                        transform=lambda x: np.expand_dims(np.asarray(x, dtype=np.float32), 0) / 255)
    mnist_loader = DataLoader(mnist_train, args.batchsize)
    mnist_loader = InfiniteDataLoader(mnist_loader)

    generator = get_generator(Z_DIM).to(device)
    critic = get_critic().to(device)

    opt_g = Adam(generator.parameters(), args.alpha, (args.beta1, args.beta2))
    opt_c = Adam(critic.parameters(), args.alpha, (args.beta1, args.beta2))

    trainer = Engine(WGANTrainer(mnist_loader, generator, critic, opt_g, opt_c,
                                 args.n_cri, args.gp_lam, device))

    log_dict = {}
    accumulator = MetricsAccumulator(["generator_loss", "critic_loss"])
    trainer.add_event_handler(Events.ITERATION_COMPLETED, accumulator)
    trainer.add_event_handler(Events.ITERATION_COMPLETED(every=500), record_metrics(log_dict, accumulator))
    trainer.add_event_handler(Events.ITERATION_COMPLETED(every=500), print_metrics(log_dict, accumulator.keys))
    trainer.add_event_handler(Events.ITERATION_COMPLETED(every=500),
                              plot_metrics(log_dict, "iteration", accumulator.keys, result_dir / "metrics.pdf"))
    trainer.add_event_handler(Events.ITERATION_COMPLETED(every=500), save_img(generator, result_dir / "generated_samples", device))

    # 指定されたイテレーション数で終了させる
    trainer.add_event_handler(Events.ITERATION_COMPLETED(once=args.iteration), lambda engine: engine.terminate())

    trainer.run(mnist_loader, max_epochs=10**10)

In [34]:
import argparse

## Modificados parámetros por defecto a los de PassGAN paper [1]
#   Falta determinar lo que definen en el paper como:
#   + Number of discriminator iterations per generator iteration: 10
#   + Model dimensionality (Dimensionalidad de cada una de las 5 Redes Residuales): 128
#   + Verificar que hay 5 redes residuales!!!
#   + Output sequence: 10 characters
#   + Size input noise: 128 floting point numbers
#   + Max nº examples: size of entire dataset
#   
#   
#   Del wgan.py falta determinar que son los parámetro: 
#   - gp_lam: gradient penalty lamda? --> + Gradient penalty coefficient (λ): 10
#   - g: si se usa GPU o no. 1 se usa si esta disponible.
#   - n_cri

parser = argparse.ArgumentParser(description="WGAN(-GP)")
parser.add_argument("-b", "--batchsize", type=int, default=64) #ok
parser.add_argument("-i", "--iteration", type=int, default=199000) #ok
parser.add_argument("--alpha", type=float, default=10**-4) #ok
parser.add_argument("--beta1", type=float, default=0.5) #ok
parser.add_argument("--beta2", type=float, default=0.9) #ok
parser.add_argument("--n_cri", type=int, nargs=2, default=[5, 100])
parser.add_argument("--gp_lam", type=float, default=10.0) #ok
parser.add_argument("-g", type=int, default=1, help="GPU ID (negative value indicates CPU mode)")
parser.add_argument("--result_dir", default="result")
args_aux = parser.parse_args(args=[])
pprint.pprint(vars(args_aux))
main(args_aux)


{'alpha': 0.0001,
 'batchsize': 64,
 'beta1': 0.5,
 'beta2': 0.9,
 'g': 1,
 'gp_lam': 10.0,
 'iteration': 199000,
 'n_cri': [5, 100],
 'result_dir': 'result'}
GPU mps for MacOS
generator_loss: 0.7859628632441163, critic_loss: -4.550230452510714
generator_loss: 0.29959071191214026, critic_loss: -1.4116484018605948
generator_loss: 2.082116430431604, critic_loss: -1.193415075558424
generator_loss: 3.4949639447331426, critic_loss: -1.2989082745870948
generator_loss: 5.341739231109619, critic_loss: -1.4048410435667635
generator_loss: 6.520712934494019, critic_loss: -1.45491073954627
generator_loss: 7.228562222003937, critic_loss: -1.5091009885473532
generator_loss: 7.551783550262451, critic_loss: -1.569752344903201
generator_loss: 7.399528996944428, critic_loss: -1.6239706043985478
generator_loss: 7.174618826389313, critic_loss: -1.6538480126886064
generator_loss: 7.287543992996216, critic_loss: -1.6591238124606005
generator_loss: 7.086586984634399, critic_loss: -1.7106633759170773
generato

Engine run is terminating due to exception: 


KeyboardInterrupt: 

In [ ]:
import torch
import math
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())

True
True
